# Notebook for End to End testing in the MID PSI

- DishLMC in the loop
- SPFRx in the loop
- TMC/SDP/CSP in the loop
- Receives and plots the visibilities to SDP

In [ ]:
import json
import os
from time import sleep
from IPython.display import clear_output
from tango import Database, DevFailed, DeviceProxy

print("Setting up parameters\n")

PSI_NAMESPACE = "ci-ska-mid-psi-1403298073-jaredmda"

TANGO_HOST = f"databaseds-tango-base.{PSI_NAMESPACE}.svc.cluster.local:10000"
os.environ["TANGO_HOST"] = TANGO_HOST

target_board = [1]  # update if necessary
RECEPTORS = ["SKA001"]  # update if necessary

# Config files setup
DATA_DIR = "../../data"
TMC_CONFIGS = f"{DATA_DIR}/psi/tmc"
ASSIGN_RESOURCES_FILE = f"{TMC_CONFIGS}/assign_resources.json"
CONFIGURE_SCAN_FILE = f"{TMC_CONFIGS}/configure_scan.json"
SCAN_FILE = f"{TMC_CONFIGS}/scan.json"
RELEASE_RESOURCES_FILE = f"{DATA_DIR}/release_resources.json"

CBF_CONFIGS = f"{DATA_DIR}/psi/cbf"
DISH_CONFIG_FILE = f"{CBF_CONFIGS}/sys_params/load_dish_config.json"

# MCS/Talon setup
COMMON_CONFIG_FOLDER = os.path.join(DATA_DIR, "psi/cbf")
HW_CONFIG_FOLDER = os.path.join(COMMON_CONFIG_FOLDER, "hw_config")
INIT_SYS_PARAM_FILE = os.path.join(COMMON_CONFIG_FOLDER, "sys_params/initial_system_param.json")

files = [
    INIT_SYS_PARAM_FILE,
    ASSIGN_RESOURCES_FILE,
    CONFIGURE_SCAN_FILE,
    SCAN_FILE,
]

# Slim config files setup
if len(target_board) == 1:
    slim_fs_config = ""
    slim_vis_config = ""
else:
    slim_fs_config = "fs_slim_4vcc_1fsp.yaml"  # update if necessary
    slim_vis_config = ""  # update if necessary

SLIM_CONFIGS = os.path.join(COMMON_CONFIG_FOLDER, "slim_config")
SLIM_FS_CONFIG_FILE = os.path.join(SLIM_CONFIGS, slim_fs_config)
SLIM_VIS_CONFIG_FILE = os.path.join(SLIM_CONFIGS, slim_vis_config)

if slim_fs_config != "":
    print("Loading custom SLIM fs config")
    !kubectl cp $SLIM_FS_CONFIG_FILE $PSI_NAMESPACE/ds-cbfcontroller-controller-0:/app/mnt/slim/fs_slim_config.yaml
    files.append(SLIM_FS_CONFIG_FILE)
else:
    print("SLIM fs will use default config")

if slim_vis_config != "":
    print("Loading custom SLIM vis config")
    !kubectl cp $SLIM_VIS_CONFIG_FILE $PSI_NAMESPACE/ds-cbfcontroller-controller-0:/app/mnt/slim/vis_slim_config.yaml
    files.append(SLIM_VIS_CONFIG_FILE)
else:
    print("SLIM vis will use default config")

print()
for file in files:
    if os.path.isfile(file):
        print(f"{file} exists: ✔️")
    else:
        print(f"{file} does not exist ❌")

print("\nLinks for Taranta and QA Display:")
print(f"https://142.73.34.170/{PSI_NAMESPACE}/signal/display/")
print(f"https://142.73.34.170/{PSI_NAMESPACE}/taranta/devices")

### MCS Deployer Setup

In [ ]:
db = Database()

deployer_tango = DeviceProxy("mid_csp_cbf/ec/deployer")
# Check the devices initially deployed to the database
print(*db.get_device_exported("*").value_string, sep="\n")

print()
if any(i > 4 for i in target_board):
    print("Using swap for higher number talons")
    config = "hw_config_swap_psi.yaml"
    print("Modifying target to use lower nums to match swap file")
    target_board = list(map(lambda x: x - 4, target_board))
else:
    print("Using standard HW config")
    config = "hw_config_psi.yaml"

HW_CONFIG_FILE = os.path.join(HW_CONFIG_FOLDER, config)
print()
if os.path.isfile(HW_CONFIG_FILE):
    print("HW config: ✔️")
else:
    print("hw config: ❌")

In [ ]:
!kubectl cp $HW_CONFIG_FILE $PSI_NAMESPACE/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml

In [ ]:
deployer_tango.targetTalons = target_board
print(f"deployer_tango.targetTalons: {deployer_tango.targetTalons}")

In [ ]:
deployer_tango.generate_config_jsons()

In [ ]:
deployer_tango.set_timeout_millis(300000)
try:
    deployer_tango.download_artifacts()
except DevFailed as e:
    print(e)
    print(
        "Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes."
    )
deployer_tango.set_timeout_millis(4500)
print("DONE")

In [ ]:
deployer_tango.configure_db()
print("DONE")

### Set Up and Configure Device Proxies 

In [ ]:
print("Setting up the device proxies")

# TMC proxies
tmc_central_node = DeviceProxy("ska_mid/tm_central/central_node")
tmc_csp_master = DeviceProxy("ska_mid/tm_leaf_node/csp_master")
tmc_csp_subarray = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray01")
tmc_subarray = DeviceProxy("ska_mid/tm_subarray_node/1")

# CSP.LMC proxies
csp_control = DeviceProxy("mid-csp/control/0")
csp_subarray = DeviceProxy("mid-csp/subarray/01")

# CBF proxies
cbf_controller = DeviceProxy("mid_csp_cbf/sub_elt/controller")
cbf_subarray = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")

# Dish Leaf proxies
dish_leaf_node_ska001 = DeviceProxy("ska_mid/tm_leaf_node/d0001")

# SDP proxies
sdp_subarray = DeviceProxy("mid-sdp/subarray/01")

# Leaf Node proxies
csp_subarray_leaf_node = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray01")
sdp_subarray_leaf_node = DeviceProxy("ska_mid/tm_leaf_node/sdp_subarray01")
csp_master_leaf_node = DeviceProxy("ska_mid/tm_leaf_node/csp_master")

# Set devices to adminMode = ONLINE
csp_control.adminMode = 0
csp_subarray.adminMode = 0

sleep(2)
print("\nChecking admin mode after setting to ONLINE (0):")
print(f"  CSP Control: {csp_control.adminMode}")
print(f"  CSP Subarray: {csp_subarray.adminMode}")
print(f"  CBF Controller: {cbf_controller.adminMode}")
print(f"  CBF Subarray: {cbf_subarray.adminMode}")

# Set CBF Simulation mode to false and CBF timeout to 99s
csp_control.cbfSimulationMode = 0
csp_control.commandTimeout = 99

sleep(2)
print("\nChecking CBF Simulation Mode and CBF Timeout:")
print(f"  CBF Simulation Mode: {bool(csp_control.cbfSimulationMode)}")
print(f"  CBF Timeout: {csp_control.commandTimeout} sec")

### Load the Dish Vcc Config / Init Sys Params

NOTE: Might need to run multiple times.

In [ ]:
with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = json.load(f)

dish_config_json["tm_data_sources"][
    0
] = "car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"
dish_config_json["tm_data_filepath"] = "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"
# dish_config_json["dish_parameters"]["SKA001"][1] = 618
print(f"dish_config_json file contents: \n{dish_config_json}")
tmc_central_node.LoadDishCfg(json.dumps(dish_config_json))

sleep(2)
print(f"TMC CSP Master's Dish Vcc Config attribute value: \n{tmc_csp_master.dishVccConfig}")
print(
    f"\nTMC CSP Master's Source Dish Vcc Config attribute value: \n{tmc_csp_master.sourceDishVccConfig}"
)

##### Use This to Check if Config is Set Before Proceeding

In [ ]:
print(f"tmc_central_node.isDishVccConfigSet: {tmc_central_node.isDishVccConfigSet}")

print(f"\nTMC CSP Master's Dish Vcc Config attribute value: \n{tmc_csp_master.dishVccConfig}")
print(
    f"\nTMC CSP Master's Source Dish Vcc Config attribute value: \n{tmc_csp_master.sourceDishVccConfig}"
)

### Turn the Telescope On

In [ ]:
print(tmc_central_node.telescopeState)

In [ ]:
print("Running the TelescopeOn command")
tmc_central_node.TelescopeOn()

while int(tmc_central_node.telescopeState) != 0:
    print("  Waiting for Telescope to come ON")
    sleep(5)

print("YAY Telecope is ON")

In [ ]:
print("Verifying the states:")
print(f"  TMC Central Node State: {tmc_central_node.State()}")
print(f"  CSP Control State: {csp_control.State()}")
print(f"  CBF Controller State: {cbf_controller.State()}")
print(f"  TMC Subarray State: {tmc_subarray.State()}")

### Assign Resources

In [ ]:
print(f"SDP Subarray State: {sdp_subarray.state()}")
print(f"CBF Subarray Receptors: {cbf_subarray.receptors}")

print(
    "\nRunning the AssignResources command: subarray obsstate should go to Idle (2) and receptor IDs should be assigned"
)

with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as f:
    assign_resources_json = json.load(f)
    assign_resources_json["dish"]["receptor_ids"] = RECEPTORS
    assign_resources_json["sdp"]["resources"]["receptors"] = RECEPTORS

print(f"\nassign_resources_json: \n{assign_resources_json}")

tmc_subarray.AssignResources(json.dumps(assign_resources_json))


In [ ]:
print(f"\nCBF Subarray Observation State: {cbf_subarray.obsState}")
print(f"TMC Subarray Observation State: {tmc_subarray.obsState}")

Slew

In [ ]:
import time
dish_manager = DeviceProxy("mid-dish/dish-manager/ska001")

dish_manager.slew([181.0,31.0])

while dish_manager.achievedpointing[1] != 181.0 or dish_manager.achievedpointing[2] != 31.0:
  time.sleep(1)

print("Done slewing")

### Configure Scan

##### Wait for SDP Pods Before Proceeding

Monitor the vis-receive pod using k9s on the -sdp namespace and ensure that all jobs are completed and the pod is Ready and in Running status before executing the next step.

If executing BITE, switch to that notebook NOW

In [ ]:
print("Running the Configure command: subarray obsstate should go to Ready (4)")

with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as f:
    configure_scan_json = json.load(f)

print(f"\nconfigure_scan_json: {configure_scan_json}")

tmc_subarray.Configure(json.dumps(configure_scan_json))

In [ ]:
print(f"\nCBF Subarray Observation State: {cbf_subarray.obsState}")
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node.cspSubarrayObsState}")
print(f"TMC Subarray Observation State: {tmc_subarray.obsState}")

### Scan

DO NOT USE FOR NOW

In [ ]:
print("Running the Scan command: subarray obsstate should go to Scanning (5)")

with open(SCAN_FILE, encoding="utf-8") as f:
    scan_json = f.read()

print(f"\nscan_json: {scan_json}")

tmc_subarray.Scan(scan_json)

sleep(2)
print(f"\nCBF Subarray Observation State: {cbf_subarray.obsState}")

TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING

In [ ]:
with open("../../data/psi/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    sdp_scan = d["sdp"]
    print(sdp_scan)
    sdp_subarray_leaf_node.scan(json.dumps(sdp_scan))

sleep(10)
print(sdp_subarray_leaf_node.sdpSubarrayObsState)

with open("../../data/psi/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    csp_scan = d["csp"]
    print(csp_scan)
    csp_subarray_leaf_node.scan(json.dumps(csp_scan))

sleep(2)
print(csp_subarray_leaf_node.cspSubarrayObsState)

### END SCAN


DO NOT USE FOR NOW SINCE TMC SUBARRAY IS STUCK IN CONFIGURING

In [ ]:
print("Running the End Scan command: subarray obsstate should go to Ready (4)")

tmc_subarray.EndScan()

sleep(2)
print(f"\nTMC Subarray Observation State: {tmc_subarray.obsState}")

TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING

In [ ]:
print("Running the End Scan command: subarray obsstate should go to Ready (4)")

sdp_subarray_leaf_node.EndScan()
sleep(2)
csp_subarray_leaf_node.EndScan()
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node.cspSubarrayObsState}")

In [ ]:
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node.cspSubarrayObsState}")

### END

DO NOT USE (TMC STUCK IN CONFIGURING)

In [ ]:
print("Running the End command: subarray obsstate should go to Idle (2)")

tmc_subarray.End()

sleep(2)
print(f"\nTMC Subarray Observation State: {cbf_subarray.obsState}")

TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING

In [ ]:
print("Running the End command: subarray obsstate should go to Idle (2)")

sdp_subarray_leaf_node.End()
sleep(2)
csp_subarray_leaf_node.End()

sleep(2)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node.cspSubarrayObsState}")

In [ ]:
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node.cspSubarrayObsState}")

### Release Resources

DO NOT USE (TMC STUCK IN CONFIGURING)

In [ ]:
print(
    "Running the Release All Resources command: subarray obsstate should go to Empty (0) and receptor IDs should be empty"
)

tmc_subarray.ReleaseAllResources()
while tmc_subarray.obsState != 0:
    sleep(5)
    clear_output(wait=True)
    print
sleep(2)
print(f"\nTMC Subarray Observation State: {tmc_subarray.obsState}")

TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING

In [ ]:
print(
    "Running the Release All Resources command: subarray obsstate should go to Empty (0) and receptor IDs should be empty"
)

sdp_subarray_leaf_node.ReleaseAllResources()
sleep(2)
csp_subarray_leaf_node.ReleaseAllResources()

while sdp_subarray_leaf_node.obsState != 0 and csp_subarray_leaf_node.obsState != 0:
    sleep(5)
    clear_output(wait=True)
    print("These should both return to EMPTY")
    print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node.sdpSubarrayObsState}")
    print(f"CSP Subarray Observation State: {csp_subarray_leaf_node.cspSubarrayObsState}")

### Telescope Off

In [ ]:
print("Running the TelescopeOff command")

tmc_central_node.TelescopeOff()

while tmc_central_node.State() != 1:
    sleep(5)
    clear_output(wait=True)
    print("These should all go to OFF")
    print(f"  TMC Central Node State: {tmc_central_node.State()}")
    print(f"  CSP Control State: {csp_control.State()}")
    print(f"  CBF Controller State: {cbf_controller.State()}")